In [ ]:
import os
path = os.getcwd()
path

In [ ]:
import pandas as pd
csv_input_path = path + "/augmented_dataset_final_outputs_with_clusters.csv"
# Read the CSV file into a DataFrame
df = pd.read_csv(csv_input_path, encoding='utf-8')

# Display the first few rows of the DataFrame to check the contents
display(df)

In [ ]:
from openai import OpenAI

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
client = OpenAI()

In [ ]:
# Reading a review which belongs to each group.
rev_per_cluster = 4
max_tokens_per_review = 1024  # Limit characters per review
n_clusters = 5

for i in range(n_clusters):
    print(f"Cluster {i} Theme:", end=" ")

    # Sample reviews and truncate each review if necessary
    reviews = df['Text'][df.cluster == i].replace("Title: ", "").replace("\n\nContent: ", ":  ")
    
    # Sample reviews while ensuring the maximum character count is respected
    sampled_reviews = reviews.sample(rev_per_cluster, random_state=42).values
    truncated_reviews = [review[:max_tokens_per_review] for review in sampled_reviews]

    reviews_text = "\n".join(truncated_reviews)

    messages = [
        {"role": "user", "content": f'Cosa hanno in comune i seguenti documenti?\n\nDocumenti Panthera:\n"""\n{reviews_text}\n"""\n\nTema:'}
    ]

    # Check the token count before making the request
    num_tokens = sum(len(review.split()) for review in truncated_reviews) + len(messages[0]['content'].split())
    
    if num_tokens > 8192:
        print(f"Warning: Message exceeds maximum token length: {num_tokens} tokens. Reduce review count or length.")
        continue

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0,
        max_tokens=512,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    
    print(response.choices[0].message.content.replace("\n", ""))

    sample_cluster_rows = df[df.cluster == i].sample(rev_per_cluster, random_state=42)

    for j in range(rev_per_cluster):
        print(sample_cluster_rows.FileName.values[j])

    print("-" * 100)

In [ ]:
for cluster in df['cluster'].unique():
    print(f"Cluster {cluster}:")
    file_names = df[df['cluster'] == cluster]['FileName'].tolist()
    for file_name in file_names:
        print(f"  - {file_name}")
    print("\n")